In [9]:
from pathlib import Path
import json
import numpy as np
import pandas as pd


In [10]:
def cdr_indices(inverse_number):
    cdr_list = []
    for cdr_border in [(25,39+1),(55,66+1),(105,119+1)]:
        left = cdr_border[0]
        right = cdr_border[1]
        while str(left) not in inverse_number :
            left+=1
        while str(right) not in inverse_number:
            right-=1
        cdr = list(range(inverse_number[str(left)], inverse_number[str(right)]))
        cdr_list.append(cdr)
    return cdr_list

def get_cdr_pm2_indices(inverse_number_heavy, inverse_number_light):
    cdr_heavy_list = cdr_indices(inverse_number_heavy)
    cdr_light_list = cdr_indices(inverse_number_light)
    cdr1h, cdr2h, cdr3h = cdr_heavy_list[0],cdr_heavy_list[1],cdr_heavy_list[2]
    cdr1l,cdr2l,cdr3l = cdr_light_list[0],cdr_light_list[1],cdr_light_list[2]
    return cdr1h, cdr2h, cdr3h, cdr1l,cdr2l,cdr3l


In [11]:
test="/home/gathenes/paragraph_benchmark/241025_expanded/test_set/dict.json"
train="/home/gathenes/paragraph_benchmark/241020_expanded/train_set/dict.json"
val="/home/gathenes/paragraph_benchmark/241020_expanded/val_set/dict.json"


In [12]:
with open(test) as f:
    test_dict=json.load(f)


In [13]:
with open(train) as f:
    train_dict=json.load(f)


In [14]:
with open(val) as f:
    val_dict=json.load(f)


In [15]:
print(test_dict["0"]["H_id numbers"])


['1', '2', '3', '4', '5', '6', '7', '8', '9', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128']


In [16]:
label_dict=[]
for type in [test_dict]:
    for values in type.values():
        pdb=values["pdb_code"]
        temp_dict={}
        temp_dict["pdb"]=pdb
        labels_heavy, labels_light = values["H_id labels 4.5"], values["L_id labels 4.5"]
        numbers_heavy, numbers_light = values["H_id numbers"], values["L_id numbers"]
        sequence_heavy, sequence_light = values["H_id sequence"], values["L_id sequence"]
        inverse_numbers_heavy = {num:i for i,num in enumerate(numbers_heavy)}
        inverse_numbers_light = {num:i for i,num in enumerate(numbers_light)}
        cdr1h,cdr2h,cdr3h,_,_,_ = get_cdr_pm2_indices(inverse_numbers_heavy, inverse_numbers_light)
        temp_dict["cdr1 label"]="".join([str(labels_heavy[each]) for each in cdr1h])
        temp_dict["cdr2 label"]="".join([str(labels_heavy[each]) for each in cdr2h])
        temp_dict["cdr3 label"]="".join([str(labels_heavy[each]) for each in cdr3h])
        temp_dict["cdr1 sequence"]="".join([str(sequence_heavy[each]) for each in cdr1h])
        temp_dict["cdr2 sequence"]="".join([str(sequence_heavy[each]) for each in cdr2h])
        temp_dict["cdr3 sequence"]="".join([str(sequence_heavy[each]) for each in cdr3h])
        label_dict.append(temp_dict)


In [17]:
paragraph=pd.DataFrame.from_records(label_dict)


In [18]:
test_paragraph = pd.read_csv("/data/Paratyping_project_data/paratope_prediction_dataset/cleaned_paper_dataset.csv")


In [19]:
test_paragraph["pdb"]=test_paragraph["pdb"].str.lower()
test_paragraph["ph1"]=test_paragraph["ph1"].apply(lambda x: x.replace(' ', ''))
test_paragraph["ph2"]=test_paragraph["ph2"].apply(lambda x: x.replace(' ', ''))
test_paragraph["ph3"]=test_paragraph["ph3"].apply(lambda x: x.replace(' ', ''))


In [20]:
for pdb in paragraph["pdb"].unique():
    if pdb in test_paragraph["pdb"].unique():
        cdr2,cdr1_seq = paragraph.query("pdb==@pdb")[["cdr3 label", "cdr3 sequence"]].values[0]
        cdr1_test, cdr1_test_seq = test_paragraph.query("pdb==@pdb")[["ph3", "cdrh3"]].values[0]
        if cdr1_test!=cdr2 or cdr1_seq!=cdr1_test_seq:
            print("-"*50)
            print(pdb)
            print(cdr2, cdr1_seq)
            print(cdr1_test,cdr1_test_seq)


--------------------------------------------------
1kcs
00111110000 ARGGTGFPYWG
00111100000 ARGGTGFPYWG
--------------------------------------------------
4bz1
001111100000 SRGSKGAMDYWG
001111110000 SRGSKGAMDYWG
--------------------------------------------------
4j6r
0000001011000000 ARGVRRDASWWLQFWG
0000011011000000 ARGVRRDASWWLQFWG
--------------------------------------------------
4nrx
0000100001111111101000000 ARQNHYGSGSYFYRTAYYYAMDVWG
0010100001111111101000000 ARQNHYGSGSYFYRTAYYYAMDVWG
--------------------------------------------------
4wht
0111000 TRMDYWG
0110000 TRMDYWG
--------------------------------------------------
4xxd
1111100 ASGDYWG
1110100 ASGDYWG
--------------------------------------------------
4ydj
0001111100011110101000000 ARERGRHFEPKNRDNLEGKFFDLWG
0011111101011111101000000 ARERGRHFEPKNRDNLEGKFFDLWG
--------------------------------------------------
5anm
001101101000 ATVMGGGYDYWG
0000000000000000 ATVMGKWIKGGYDYWG
--------------------------------------------------
5

# trash

In [21]:
from Levenshtein import distance
distance_list=[]
for pdb, label in pdb_to_labels.items():
    if f"{pdb}_heavy" in label_dict:
        print(pdb,label)
        print(pdb,label_dict[f"{pdb}_heavy"])
        dis = distance(label,label_dict[f"{pdb}_heavy"])
        print(dis)
        distance_list.append(dis/len(label))
        print("-----------")


NameError: name 'pdb_to_labels' is not defined

In [ ]:
print(np.mean(distance_list))
print(len(distance_list))


nan
0


/home/gathenes/.pyenv/versions/3.9.6/envs/analysis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/gathenes/.pyenv/versions/3.9.6/envs/analysis/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
pdb_to_labels=test_paragraph.set_index("pdb")["heavy labels"].to_dict()


KeyError: 'heavy labels'

In [ ]:
test_paragraph["heavy labels"]=test_paragraph["ph1"]+test_paragraph["ph2"]+test_paragraph["ph3"]
